In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

Exception ignored in: <function _xla_gc_callback at 0x78e58ef789d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [2]:
df = pd.read_csv('Small.csv', index_col=False)
texts = df['Tweet'].astype(str)
labels = df['Sarcasm'].map({'yes': 1, 'no': 0})

In [3]:
def predict_sarcasm(text, tokenizer, model, max_len):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded_sequence)
    if prediction > 0.5:
        return "This text is predicted to be sarcastic."
    else:
        return "This text is predicted to be non-sarcastic."

In [4]:
max_len = 128
vocab_size = 10000
embedding_dim = 512

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

padded_sequences = pad_sequences(sequences, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))



model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score (weighted): {f1:.4f}")

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/10
174/174 [==============================] - 250s 1s/step - loss: 0.3715 - accuracy: 0.5222 - val_loss: 0.2597 - val_accuracy: 0.5325
Epoch 2/10
174/174 [==============================] - 236s 1s/step - loss: 0.1619 - accuracy: 0.5222 - val_loss: 0.2643 - val_accuracy: 0.5325
Epoch 3/10
174/174 [==============================] - 246s 1s/step - loss: 0.0756 - accuracy: 0.5222 - val_loss: 0.3626 - val_accuracy: 0.5325
Epoch 4/10
174/174 [==============================] - 247s 1s/step - loss: 0.0423 - accuracy: 0.5222 - val_loss: 0.3662 - val_accuracy: 0.5325
Epoch 5/10
174/174 [==============================] - 246s 1s/step - loss: 0.0284 - accuracy: 0.5222 - val_loss: 0.3891 - val_accuracy: 0.5325
Epoch 6/10
44/44 [==============================] - 14s 295ms/step
F1 Score (weighted): 0.3700
44/44 [==============================] - 13s 300ms/step - loss: 0.4346 - accuracy: 0.5325
Test Loss: 0.4346, Accuracy: 0.5325


In [5]:
new_text = "My name is Tanisha."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 84ms/step
This text is predicted to be sarcastic.


In [6]:
new_text = "My mom asked me this question as well."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 167ms/step
This text is predicted to be sarcastic.


In [7]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 83ms/step
This text is predicted to be sarcastic.


In [8]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 103ms/step
This text is predicted to be sarcastic.
